# Solving SAT problem with Grover's algorithm

The **"Solving SAT problem with Grover's algorithm"** quantum kata is a series of exercises designed
to get you comfortable with using Grover's algorithm to solve realistic problems
using boolean satisfiability problems (SAT) as an example.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Oracles for SAT problems

The most interesting part of learning Grover's algorithm is solving realistic problems.
This means using oracles which express an actual problem and not simply hard-code a known solution.
In this section we'll learn how to express boolean satisfiability problems as quantum oracles.

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [1]:
%kata T11_Oracle_And 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.1.-The-AND-oracle:-$f(x)-=-x_0-\wedge-x_1$).*

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [2]:
%kata T12_Oracle_Or 

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    within {
        ApplyToEachA(X, queryRegister);
    }
    apply {
        Oracle_And(queryRegister, target);
    }
    
    X(target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.2.-The-OR-oracle:-$f(x)-=-x_0-\vee-x_1$).*

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [3]:
%kata T13_Oracle_Xor 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // For two bits, the mask would be |01> or |10>
    // For three bits, |001>, |010>, |100>, |111>
    // For four bits: |0001>, |0010>, |0100>, |1000>, |0111>, |1011>, |1110>
    // Overall pattern is "odd number of 1s".
    for i in 0 .. Length(queryRegister) - 1 {
        Controlled X([queryRegister[i]], target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.3.-The-XOR-oracle:-$f(x)-=-x_0-\oplus-x_1$).*

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

> This oracle marks two states similar to the state explored in task 1.2 of the GroversAlgorithm kata: 
$|10101...\rangle$ and $|01010...\rangle$.  
It is possible (and quite straightforward) to implement this oracle based on this observation; 
however, for the purposes of learning to write oracles to solve SAT problems we recommend using the representation above.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  Remember that you can use operations defined in previous tasks.
</details>

In [4]:
%kata T14_Oracle_AlternatingBits 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    use anc = Qubit[Length(queryRegister) - 1];
    
    within {
        // anc[i] = queryRegister[i] XOR queryRegister[i + 1]
        for i in 0 .. Length(queryRegister) - 2 {
            Oracle_Xor([queryRegister[i], queryRegister[i + 1]], anc[i]);
        }
    }
    apply {
        Oracle_And(anc, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.4.-Alternating-bits-oracle:-$f(x)-=-(x_0-\oplus-x_1)-\wedge-(x_1-\oplus-x_2)-\wedge-\dots-\wedge-(x_{N-2}-\oplus-x_{N-1})$).*

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{k},\text{ where }y_{k} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[(0, true), (1, false)]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [5]:
function GetIndex(i: Int, b : Bool) : Int {
    return i;
} 

GetIndex

In [6]:
function GetBool(i: Int, b: Bool) : Bool {
    return b;
}

GetBool

In [7]:
%kata T15_Oracle_SATClause

open Microsoft.Quantum.Arrays;

operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    let indices = Mapped(GetIndex, clause);
    
    within {
        for i in 0 .. Length(clause) - 1 {
            let idx = GetIndex(clause[i]);
            let b = GetBool(clause[i]);
            if (not b) {
                X(queryRegister[idx]);
            }
        }
    }
    apply {
        Oracle_Or(Subarray(indices, queryRegister), target);
    }
}

Testing SAT clause instance ¬x2...
Testing SAT clause instance ¬x0 ∨ x2...
Testing SAT clause instance ¬x2 ∨ ¬x0 ∨ x1...
Testing SAT clause instance ¬x3 ∨ x1 ∨ ¬x0 ∨ x2...
Testing SAT clause instance x0 ∨ x1 ∨ ¬x2...
Testing SAT clause instance x2 ∨ ¬x4 ∨ ¬x1 ∨ ¬x3 ∨ ¬x0...
Testing SAT clause instance ¬x2 ∨ x5 ∨ x4 ∨ x3 ∨ x0 ∨ ¬x1...
Testing SAT clause instance x2 ∨ ¬x4 ∨ x1 ∨ x0 ∨ ¬x3...
Testing SAT clause instance x0 ∨ ¬x1 ∨ ¬x2...
Testing SAT clause instance x0 ∨ x2 ∨ x3 ∨ ¬x1...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.5.-Evaluate-one-clause-of-a-SAT-formula).*

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [8]:
%kata T16_Oracle_SAT 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    use anc = Qubit[Length(problem)];
    
    // anc[i] is an application of the ith clause.
    within {
        for i in 0 .. Length(problem) - 1 {
            Oracle_SATClause(queryRegister, anc[i], problem[i]);
        }
    }
    apply {
        Oracle_And(anc, target);
    }
}

Testing 2-SAT instance (2, (x0 ∨ x1))...
Testing 2-SAT instance (2, (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0))...
Testing 2-SAT instance (3, (¬x1 ∨ ¬x2) ∧ (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0) ∧ (x2 ∨ x1))...
Testing k-SAT instance (3, (¬x1 ∨ ¬x2) ∧ (¬x2))...
Testing k-SAT instance (4, (¬x0) ∧ (¬x1 ∨ x3) ∧ (¬x1 ∨ ¬x0))...
Testing k-SAT instance (5, (¬x1 ∨ x4 ∨ x3 ∨ ¬x0) ∧ (¬x4 ∨ x3 ∨ ¬x0 ∨ ¬x1) ∧ (x1 ∨ x0) ∧ (¬x3 ∨ x1 ∨ x4))...
Testing k-SAT instance (6, (x1 ∨ ¬x5 ∨ ¬x4) ∧ (x3 ∨ ¬x5 ∨ ¬x2 ∨ x0) ∧ (¬x0 ∨ x1 ∨ ¬x2) ∧ (¬x0) ∧ (x1 ∨ ¬x4 ∨ ¬x0 ∨ ¬x5))...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.6.-k-SAT-problem-oracle).*

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [9]:
function Indicator(i : Int, j : Int) : Bool {
    return i == j;
}

Indicator

In [10]:
%kata T21_Oracle_Exactly1One 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    use anc = Qubit[N]; 
    // anc[i] stores the result of x_i being true, and x_j being false for all j <> i
    
    within {
        for i in 0 .. N - 1 {
            let bitMask = Mapped(Indicator(i, _), SequenceI(0, N - 1));
            ApplyControlledOnBitString(bitMask, X, queryRegister, anc[i]);
        }
    }
    apply {
        Oracle_Or(anc, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.1.-"Exactly-one-$|1\rangle$"-oracle).*

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the 3-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in task 1.6;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true, which are solutions for the general SAT problem.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Can you reuse parts of the code in section 1?
</details>

In [11]:
%kata T22_Oracle_Exactly1SAT 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    let N = Length(problem);
    use anc = Qubit[N];
    // anc[i] has the answer to the ith problem.
    within {
        for i in 0 .. N - 1 {
            for (idx, b) in problem[i] {
                if (not b) {
                   X(queryRegister[idx]);
                }
            }
            
            Oracle_Exactly1One(queryRegister, anc[i]);
            
            for (idx, b) in problem[i] {
                if (not b) {
                   X(queryRegister[idx]);
                }
            }
        }
    }
    apply {
        Oracle_And(anc, target);
    }
}

Testing exactly-1 3-SAT instance (2, (¬x1 ∨ x0))...
Testing exactly-1 3-SAT instance (3, (x1 ∨ x0 ∨ x2) ∧ (x2 ∨ ¬x1 ∨ ¬x0))...
Testing exactly-1 3-SAT instance (4, (x2 ∨ x0 ∨ x3) ∧ (¬x2 ∨ x3 ∨ x1) ∧ (¬x3 ∨ ¬x1 ∨ ¬x0))...
Testing exactly-1 3-SAT instance (5, (¬x2 ∨ ¬x0 ∨ x3) ∧ (¬x4 ∨ x2 ∨ ¬x0) ∧ (¬x4 ∨ x2 ∨ ¬x3) ∧ (¬x0 ∨ x3 ∨ x2))...
Testing exactly-1 3-SAT instance (6, (¬x5 ∨ ¬x4 ∨ ¬x3) ∧ (x5 ∨ x2 ∨ x1) ∧ (x4 ∨ x3 ∨ ¬x1) ∧ (¬x3 ∨ ¬x2 ∨ ¬x0) ∧ (¬x2 ∨ x5 ∨ x4))...


Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.2.-"Exactly-1-3-SAT"-oracle).*

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

> If you want to learn the Grover's algorithm itself, try doing [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) first.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b></summary>
Experiment with SAT instances with different number of solutions and the number of algorithm iterations 
to see how the probability of the algorithm finding the correct answer changes depending on these two factors.

For example, 
* the AND oracle from task 1.1 has exactly one solution,
* the alternating bits oracle from task 1.4 has exactly two solutions,
* the OR oracle from task 1.2 for 2 qubits has exactly 3 solutions, and so on.
</details>

In [12]:
operation ApplyMarkingOracleAsPhaseOracle (markingOracle: ((Qubit[], Qubit) => Unit is Adj), 
                                           qubits: Qubit[]) : Unit is Adj {
    use y = Qubit();
    
    within {
        X(y);
        H(y);
    }
    apply {
        markingOracle(qubits, y);
    }
}

ApplyMarkingOracleAsPhaseOracle

In [13]:
function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return ApplyMarkingOracleAsPhaseOracle(markingOracle, _);
}

OracleConverter

In [14]:
operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    use anc = Qubit();
    within {
        X(anc);
        ApplyToEachA(X, register);
    }
    apply {
        Controlled Z(register, anc);
    }
}

ConditionalPhaseFlip

In [15]:
operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    // Apply the Oracle.
    oracle(register);
    
    // Apply the Hadamard transform
    ApplyToEachA(H, register);
    
    // Perform a conditional phase shift.
    ConditionalPhaseFlip(register);
    
    // Apply the Hadamard transformation again.
    ApplyToEachA(H, register);
}

GroverIteration

In [16]:
operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    let phaseOracle = OracleConverter(oracle);
    
    ApplyToEachA(H, register);
    for i in 1 .. iterations {
        GroverIteration(register, phaseOracle);
    }
}

GroversSearch

In [24]:
operation Run_GroversSearch_Algorithm_Impl(N: Int, oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    use register = Qubit[N];
    GroversSearch(register, oracle, iterations);
    
    for i in 0 .. N - 1 {
        let result = M(register[i]);
        if (result == Zero) {
            Message($"register[{i}] = 0");
        }
        else {
            Message($"register[{i}] = 1");
        }
    }
}

Run_GroversSearch_Algorithm_Impl

In [26]:
operation Oracle_SATClause_Impl (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let clause = [(3, false), (1, true), (0, false), (2, true)];
    
    Oracle_SATClause(queryRegister, target, clause);
}

Oracle_SATClause_Impl

In [34]:
operation Oracle_SAT_Impl (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let problem = [[(0, false)], [(1, false), (3, true)], [(1, false), (0, false)]];
    
    Oracle_SAT(queryRegister, target, problem);
}

Oracle_SAT_Impl

In [45]:
operation Oracle_Exactly1_3SAT_Impl (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // (4, (x2 ∨ x0 ∨ x3) ∧ (¬x2 ∨ x3 ∨ x1) ∧ (¬x3 ∨ ¬x1 ∨ ¬x0)).
    let problem = [[(2, true), (0, true), (3, true)], [(2, false), (3, true), (1, true)], [(3, false), (1, false), (0, true)]];
    Oracle_Exactly1_3SAT(queryRegister, target, problem);
}

Oracle_Exactly1_3SAT_Impl

In [47]:
operation Run_GroversSearch_Algorithm () : Unit {
    // Optimal num iterations = approx (pi/4) * sqrt(M/N)
    // Where N is the size of the problem space and M is the number of correct solutions.
    
    Message("And Oracle:");
    // And Oracle: optimal iterations approx. pi/4 * 4
    Run_GroversSearch_Algorithm_Impl(4, Oracle_And, 3);
    
    Message("Or Oracle:");
    // Or Oracle: optimal iterations approx. pi/4 * sqrt(16/15) = 1
    Run_GroversSearch_Algorithm_Impl(4, Oracle_Or, 2);
    
    Message("XOR Oracle:");
    // Xor Oracle: correct answers are:
    // |0001>, |0010>, |0100>, |1000>
    // |0111>, |1011>, |1101>, |1110>
    Run_GroversSearch_Algorithm_Impl(4, Oracle_Xor, 2);
    
    Message("Alternating Bits Oracle:");
    // Alternating bits oracle: correct answers are:
    // |1010>, |0101>
    Run_GroversSearch_Algorithm_Impl(4, Oracle_AlternatingBits, 2);
    
    Message("SAT Clause Oracle:");
    // Oracle_SAT_Clause for ¬x3 ∨ x1 ∨ ¬x0 ∨ x2
    Run_GroversSearch_Algorithm_Impl(4, Oracle_SATClause_Impl, 4);
    
    Message("Oracle_SAT for (4, (¬x0) ∧ (¬x1 ∨ x3) ∧ (¬x1 ∨ ¬x0))");
    Run_GroversSearch_Algorithm_Impl(4, Oracle_SAT_Impl, 4);
    
    Message("Oracle_Exactly1One");
    Run_GroversSearch_Algorithm_Impl(4, Oracle_Exactly1One, 4);
    
    Message("Oracle_Exactly1_3SAT");
    // (4, (x2 ∨ x0 ∨ x3) ∧ (¬x2 ∨ x3 ∨ x1) ∧ (¬x3 ∨ ¬x1 ∨ ¬x0)).
    Run_GroversSearch_Algorithm_Impl(4, Oracle_Exactly1_3SAT_Impl, 4);
}

Run_GroversSearch_Algorithm

In [ ]:
%simulate Run_GroversSearch_Algorithm

And Oracle:
register[0] = 1
register[1] = 1
register[2] = 1
register[3] = 1
Or Oracle:
register[0] = 0
register[1] = 0
register[2] = 0
register[3] = 0
XOR Oracle:
register[0] = 1
register[1] = 0
register[2] = 0
register[3] = 0
Alternating Bits Oracle:
register[0] = 1
register[1] = 0
register[2] = 1
register[3] = 0
SAT Clause Oracle:
register[0] = 0
register[1] = 0
register[2] = 1
register[3] = 0
Oracle_SAT for (4, (¬x0) ∧ (¬x1 ∨ x3) ∧ (¬x1 ∨ ¬x0))
register[0] = 1
register[1] = 0
register[2] = 0
register[3] = 0
Oracle_Exactly1One
register[0] = 1
register[1] = 0
register[2] = 0
register[3] = 0
Oracle_Exactly1_3SAT
register[0] = 1
register[1] = 0
register[2] = 1
register[3] = 1


()

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.1.-Using-Grover's-algorithm).*

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. The number of qubits N.

  2. A marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

> Note that the similar task in the [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) required you to implement Grover's algorithm 
in a way that would be robust to accidental failures, but you knew the optimal number of iterations 
(the number that minimized the probability of such failure).
> 
> In this task you also need to make your implementation robust to not knowing the optimal number of iterations.
> You can see an example of doing that in [Exploring Grover's Algorithm tutorial](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithm.ipynb).

In [53]:
%kata T32_UniversalGroversAlgorithm 

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    // In this task you don't know the optimal number of iterations upfront, 
    // so it makes sense to try different numbers of iterations.
    // This way, even if you don't hit the "correct" number of iterations on one of your tries,
    // you'll eventually get a high enough success probability.

    // This solution tries numbers of iterations that are powers of 2;
    // this is not the only valid solution, since a lot of sequences will eventually yield the answer.
    mutable answer = [false, size = N];
    use (register, output) = (Qubit[N], Qubit());
    mutable correct = false;
    mutable iter = 1;
    repeat {
        Message($"Trying search with {iter} iterations");
        GroversSearch (register, oracle, iter);
        let res = MultiM(register);
        // to check whether the result is correct, apply the oracle to the register plus ancilla after measurement
        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            set answer = ResultArrayAsBoolArray(res);
        }
        ResetAll(register);
    } until (correct or iter > 100)  // the fail-safe to avoid going into an infinite loop
    fixup {
        set iter *= 2;
    }
    if (not correct) {
        fail "Failed to find an answer";
    }
    Message($"{answer}");
    return answer;
}

Trying search with 1 iterations
[True,True]
Trying search with 1 iterations
[True,False]
Trying search with 1 iterations
Trying search with 2 iterations
Trying search with 4 iterations
Trying search with 8 iterations
Trying search with 16 iterations
Trying search with 32 iterations
Trying search with 64 iterations
Trying search with 128 iterations
[False,True]
Trying search with 1 iterations
[False,True,False,True]
Trying search with 1 iterations
Trying search with 2 iterations
[True,False,True,False,False,True]


Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.2.-Universal-implementation-of-Grover's-algorithm).*